In [ ]:
from datapile import HuggingfaceLoader
data = HuggingfaceLoader(
    dataset_path="/home/user/ckwan1/ml/huggingface_dataset", 
    batch_size=1, 
    shuffle=False,
    num_workers=1,
)
data.setup("validation")
val_iter = iter(data.val_dataloader())
za_tensor, fastpm_tensor = next(val_iter)


/home/user/ckwan1/.conda/envs/d3m/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from unet import UNet3D, BasicBlock
from torchinfo import summary

UNet3D = UNet3D(BasicBlock, num_layers=4, base_filters=64, blocks_per_layer=2, 
                 init_dim=3)
x_shape = (1 ,3 ,32 ,32 ,32)
summary(UNet3D, x_shape)

Layer (type:depth-idx)                   Output Shape              Param #
UNet3D                                   [1, 3, 32, 32, 32]        --
├─Sequential: 1-1                        [1, 64, 32, 32, 32]       --
│    └─BasicBlock: 2-1                   [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-1                  [1, 64, 32, 32, 32]       5,248
│    │    └─BatchNorm3d: 3-2             [1, 64, 32, 32, 32]       128
│    │    └─ReLU: 3-3                    [1, 64, 32, 32, 32]       --
│    └─BasicBlock: 2-2                   [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-4                  [1, 64, 32, 32, 32]       110,656
│    │    └─BatchNorm3d: 3-5             [1, 64, 32, 32, 32]       128
│    │    └─ReLU: 3-6                    [1, 64, 32, 32, 32]       --
├─ModuleList: 1-2                        --                        --
│    └─Sequential: 2-3                   [1, 128, 16, 16, 16]      --
│    │    └─BasicBlock: 3-7              [1, 128, 16, 16, 16]      221,568


In [ ]:
from unet import StyledUNet3D
from torchinfo import summary
import torch
StyledUNet3D = StyledUNet3D(style_size = 5, num_layers=4, base_filters=64, blocks_per_layer=2, 
                 init_dim=3)
x_shape = (1 ,3 ,64 ,64 ,64)
style = (1, 5)
x_tensor = torch.randn(x_shape)
style_tensor = torch.randn(style)
summary(StyledUNet3D, input_data=[x_tensor, style_tensor])

Layer (type:depth-idx)                        Output Shape              Param #
StyledUNet3D                                  [1, 3, 64, 64, 64]        --
├─StyledSequential: 1-1                       [1, 64, 64, 64, 64]       --
│    └─StyledBlock: 2-1                       [1, 64, 64, 64, 64]       --
│    │    └─ConvStyled3d_unet: 3-1            [1, 64, 64, 64, 64]       5,266
│    │    └─BatchNorm3d: 3-2                  [1, 64, 64, 64, 64]       128
│    │    └─ReLU: 3-3                         [1, 64, 64, 64, 64]       --
│    └─StyledBlock: 2-2                       [1, 64, 64, 64, 64]       --
│    │    └─ConvStyled3d_unet: 3-4            [1, 64, 64, 64, 64]       111,040
│    │    └─BatchNorm3d: 3-5                  [1, 64, 64, 64, 64]       128
│    │    └─ReLU: 3-6                         [1, 64, 64, 64, 64]       --
├─ModuleList: 1-2                             --                        --
│    └─StyledSequential: 2-3                  [1, 128, 32, 32, 32]      --
│    │    

: 

In [2]:
from unet import StyledUNet3D
import torch
# Instantiate your model with typical parameters
model = StyledUNet3D(style_size=5, num_layers=2, base_filters=16, blocks_per_layer=1, init_dim=3)

found_trainable_param = False
print("Model Parameters:")
for name, param in model.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}, Requires Grad: {param.requires_grad}")
    if param.requires_grad:
        found_trainable_param = True

if not found_trainable_param:
    print("\nERROR: No trainable parameters found in the model!")
    # If this happens, check your __init__ args for StyledUNet3D or if any param is 0-dim.
else:
    print("\nOK: Model has trainable parameters.")

# Try a dummy forward pass (optional, but good for catching shape errors)
if found_trainable_param:
    try:
        print("\nAttempting dummy forward pass...")
        # Adjust D,H,W if necessary for your architecture
        dummy_input = torch.randn(1, 3, 32, 32, 32) # Batch, Channels, D, H, W 
        dummy_style = torch.randn(1, 5)
        output = model(dummy_input, dummy_style)
        print(f"Dummy forward pass successful! Output shape: {output.shape}")
    except Exception as e:
        print(f"Error during dummy forward pass: {e}")
        import traceback
        traceback.print_exc()

Model Parameters:
Name: encoders.0.0.conv.style_weight, Shape: torch.Size([16, 5]), Requires Grad: True
Name: encoders.0.0.conv.style_bias, Shape: torch.Size([16]), Requires Grad: True
Name: encoders.0.0.conv.weight, Shape: torch.Size([32, 16, 3, 3, 3]), Requires Grad: True
Name: encoders.0.0.conv.bias, Shape: torch.Size([32]), Requires Grad: True
Name: encoders.0.0.bn.weight, Shape: torch.Size([32]), Requires Grad: True
Name: encoders.0.0.bn.bias, Shape: torch.Size([32]), Requires Grad: True
Name: encoders.1.0.conv.style_weight, Shape: torch.Size([32, 5]), Requires Grad: True
Name: encoders.1.0.conv.style_bias, Shape: torch.Size([32]), Requires Grad: True
Name: encoders.1.0.conv.weight, Shape: torch.Size([32, 32, 3, 3, 3]), Requires Grad: True
Name: encoders.1.0.conv.bias, Shape: torch.Size([32]), Requires Grad: True
Name: encoders.1.0.bn.weight, Shape: torch.Size([32]), Requires Grad: True
Name: encoders.1.0.bn.bias, Shape: torch.Size([32]), Requires Grad: True
Name: encoders.2.0.con

In [3]:
from unet import StyledUNet3D
import torch
# Instantiate your model with typical parameters
model = StyledUNet3D(style_size=5, num_layers=2, base_filters=16, blocks_per_layer=1, init_dim=3)
print(sum(p.numel() for p in model.parameters()))  # Should be > 0
print([n for n, p in model.named_parameters() if p.requires_grad])

349893
['encoders.0.0.conv.style_weight', 'encoders.0.0.conv.style_bias', 'encoders.0.0.conv.weight', 'encoders.0.0.conv.bias', 'encoders.0.0.bn.weight', 'encoders.0.0.bn.bias', 'encoders.1.0.conv.style_weight', 'encoders.1.0.conv.style_bias', 'encoders.1.0.conv.weight', 'encoders.1.0.conv.bias', 'encoders.1.0.bn.weight', 'encoders.1.0.bn.bias', 'encoders.2.0.conv.style_weight', 'encoders.2.0.conv.style_bias', 'encoders.2.0.conv.weight', 'encoders.2.0.conv.bias', 'encoders.2.0.bn.weight', 'encoders.2.0.bn.bias', 'encoders.3.0.conv.style_weight', 'encoders.3.0.conv.style_bias', 'encoders.3.0.conv.weight', 'encoders.3.0.conv.bias', 'encoders.3.0.bn.weight', 'encoders.3.0.bn.bias', 'decoders.0.style_weight', 'decoders.0.style_bias', 'decoders.0.weight', 'decoders.0.bias', 'decoders.1.0.conv.style_weight', 'decoders.1.0.conv.style_bias', 'decoders.1.0.conv.weight', 'decoders.1.0.conv.bias', 'decoders.1.0.bn.weight', 'decoders.1.0.bn.bias', 'decoders.2.style_weight', 'decoders.2.style_bias'

In [1]:
from unet import FiLMUNet3D
from torchinfo import summary
import torch
FiLMUNet3D = FiLMUNet3D(style_size = 5, num_layers=4, base_filters=64, blocks_per_layer=2, 
                 init_dim=3)
x_shape = (1 ,3 ,32 ,32 ,32)
style = (1, 5)
x_tensor = torch.randn(x_shape)
style_tensor = torch.randn(style)
summary(FiLMUNet3D, input_data=[x_tensor, style_tensor])

Layer (type:depth-idx)                   Output Shape              Param #
FiLMUNet3D                               [1, 3, 32, 32, 32]        --
├─StyledSequential: 1-1                  [1, 64, 32, 32, 32]       --
│    └─FiLMBlock: 2-1                    [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-1                  [1, 64, 32, 32, 32]       5,248
│    │    └─BatchNorm3d: 3-2             [1, 64, 32, 32, 32]       128
│    │    └─FiLMLayer3D: 3-3             [1, 64, 32, 32, 32]       768
│    │    └─ReLU: 3-4                    [1, 64, 32, 32, 32]       --
│    └─FiLMBlock: 2-2                    [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-5                  [1, 64, 32, 32, 32]       110,656
│    │    └─BatchNorm3d: 3-6             [1, 64, 32, 32, 32]       128
│    │    └─FiLMLayer3D: 3-7             [1, 64, 32, 32, 32]       768
│    │    └─ReLU: 3-8                    [1, 64, 32, 32, 32]       --
├─ModuleList: 1-2                        --                        --
│  

In [13]:
from style_vnet import StyledVNet
from torchinfo import summary
import torch
StyledVNet = StyledVNet(style_size = 5, in_chan=3, out_chan=3)
x_shape = (1 ,3 ,64 ,64 ,64)
style = (1, 5)
x_tensor = torch.randn(x_shape)
style_tensor = torch.randn(style)
summary(StyledVNet, input_data=[x_tensor, style_tensor])

Layer (type:depth-idx)                   Output Shape              Param #
StyledVNet                               [1, 3, 24, 24, 24]        --
├─ResStyledBlock: 1-1                    [1, 64, 60, 60, 60]       --
│    └─ConvStyled3d: 2-1                 [1, 64, 64, 64, 64]       274
│    └─ModuleList: 2-2                   --                        --
│    │    └─ConvStyled3d: 3-1            [1, 64, 62, 62, 62]       5,266
│    │    └─LeakyReLUStyled: 3-2         [1, 64, 62, 62, 62]       --
│    │    └─ConvStyled3d: 3-3            [1, 64, 60, 60, 60]       111,040
│    │    └─BatchNormStyled3d: 3-4       [1, 64, 60, 60, 60]       128
│    └─LeakyReLUStyled: 2-3              [1, 64, 60, 60, 60]       --
├─ConvStyledBlock: 1-2                   [1, 64, 30, 30, 30]       --
│    └─ModuleList: 2-4                   --                        --
│    │    └─ConvStyled3d: 3-5            [1, 64, 30, 30, 30]       33,216
│    │    └─BatchNormStyled3d: 3-6       [1, 64, 30, 30, 30]       128


In [1]:
from style_vnet import GeneralizedStyledVNet
from torchinfo import summary
import torch
GeneralizedStyledVNet = GeneralizedStyledVNet(style_size = 5, in_chan=3, out_chan=3)
x_shape = (1 ,3 ,64 ,64 ,64)
style = (1, 5)
x_tensor = torch.randn(x_shape)
style_tensor = torch.randn(style)
summary(GeneralizedStyledVNet, input_data=[x_tensor, style_tensor])

Layer (type:depth-idx)                        Output Shape              Param #
GeneralizedStyledVNet                         [1, 3, 24, 24, 24]        --
├─ModuleList: 1-3                             --                        (recursive)
│    └─ResStyledBlock: 2-1                    [1, 64, 60, 60, 60]       --
│    │    └─ConvStyled3d: 3-1                 [1, 64, 64, 64, 64]       274
│    │    └─ModuleList: 3-2                   --                        116,434
│    │    └─LeakyReLUStyled: 3-3              [1, 64, 60, 60, 60]       --
├─ModuleList: 1-4                             --                        (recursive)
│    └─ConvStyledBlock: 2-2                   [1, 64, 30, 30, 30]       --
│    │    └─ModuleList: 3-4                   --                        33,344
├─ModuleList: 1-3                             --                        (recursive)
│    └─ResStyledBlock: 2-3                    [1, 64, 26, 26, 26]       --
│    │    └─ConvStyled3d: 3-5                 [1, 64, 30, 

In [10]:
from naf_net import NAFNet3D_base
from torchinfo import summary
import torch
NAFNet3D_base = NAFNet3D_base(width=64,enc_blk_nums=[2,2,4,8],middle_blk_num=6,dec_blk_nums=[2,2,2,2],dw_expand=2,ffn_expand=2)
x_shape = (1 ,3 ,32 ,32 ,32)
x_tensor = torch.randn(x_shape)
summary(NAFNet3D_base, input_data=x_tensor)

Layer (type:depth-idx)                             Output Shape              Param #
NAFNet3D_base                                      [1, 3, 32, 32, 32]        --
├─Conv3d: 1-1                                      [1, 64, 32, 32, 32]       5,248
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-1                             [1, 64, 32, 32, 32]       --
│    │    └─BaselineBlock: 3-1                     [1, 64, 32, 32, 32]       53,696
│    │    └─BaselineBlock: 3-2                     [1, 64, 32, 32, 32]       53,696
├─ModuleList: 1-9                                  --                        (recursive)
│    └─Conv3d: 2-2                                 [1, 128, 16, 16, 16]      65,664
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-3                             [1, 128, 16, 16, 16]      --
│    │    └─BaselineBlock: 3-3                     [1, 128, 16, 16, 16]  

In [ ]:
from naf_net import NAFNet3D
from torchinfo import summary
import torch
NAFNet3D = NAFNet3D(width=64,enc_blk_nums=[2,2,4,8],middle_blk_num=4,dec_blk_nums=[2,2,2,2])
x_shape = (1 ,3 ,32 ,32 ,32)
x_tensor = torch.randn(x_shape)
summary(NAFNet3D, input_data=x_tensor)

Layer (type:depth-idx)                             Output Shape              Param #
NAFNet3D                                           [1, 3, 32, 32, 32]        --
├─Conv3d: 1-1                                      [1, 64, 32, 32, 32]       5,248
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-1                             [1, 64, 32, 32, 32]       --
│    │    └─BaselineBlock: 3-1                     [1, 64, 32, 32, 32]       53,696
│    │    └─BaselineBlock: 3-2                     [1, 64, 32, 32, 32]       53,696
├─ModuleList: 1-9                                  --                        (recursive)
│    └─Conv3d: 2-2                                 [1, 128, 16, 16, 16]      65,664
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-3                             [1, 128, 16, 16, 16]      --
│    │    └─BaselineBlock: 3-3                     [1, 128, 16, 16, 16]  

: 

In [1]:
from naf_net import NAFNet3D_modulated, BaselineBlock_SCA_FullyModulated, BaselineBlock_SCA_Modulated
from torchinfo import summary
import torch
NAFNet3D = NAFNet3D_modulated(width=64,enc_blk_nums=[2,2,4,8],middle_blk_num=4,dec_blk_nums=[2,2,2,2],block_type = BaselineBlock_SCA_FullyModulated,style_dim=5, modulate_outer_convs=False)
x_shape = (1 ,3 ,32 ,32 ,32)
s_shape = (1, 5)
x_tensor = torch.randn(x_shape)
s_tensor = torch.randn(s_shape)
summary(NAFNet3D, input_data=[x_tensor,s_tensor])

Layer (type:depth-idx)                                  Output Shape              Param #
NAFNet3D_modulated                                      [1, 3, 32, 32, 32]        --
├─Conv3d: 1-1                                           [1, 64, 32, 32, 32]       5,248
├─ModuleList: 1-8                                       --                        (recursive)
│    └─Sequential: 2-1                                  --                        --
│    │    └─BaselineBlock_SCA_FullyModulated: 3-1       [1, 64, 32, 32, 32]       57,472
│    │    └─BaselineBlock_SCA_FullyModulated: 3-2       [1, 64, 32, 32, 32]       57,472
├─ModuleList: 1-9                                       --                        (recursive)
│    └─Conv3d: 2-2                                      [1, 128, 16, 16, 16]      65,664
├─ModuleList: 1-8                                       --                        (recursive)
│    └─Sequential: 2-3                                  --                        --
│    │    └─Baseli

In [ ]:
for name, param in NAFNet3D.named_parameters():
    print(name)

intro.weight
intro.bias
encoders.0.0.beta
encoders.0.0.gamma
encoders.0.0.mod_conv1.style_weight
encoders.0.0.mod_conv1.style_bias
encoders.0.0.mod_conv1.weight
encoders.0.0.mod_conv1.bias
encoders.0.0.mod_conv2.style_weight
encoders.0.0.mod_conv2.style_bias
encoders.0.0.mod_conv2.weight
encoders.0.0.mod_conv2.bias
encoders.0.0.mod_conv3.style_weight
encoders.0.0.mod_conv3.style_bias
encoders.0.0.mod_conv3.weight
encoders.0.0.mod_conv3.bias
encoders.0.0.sca_conv.style_weight
encoders.0.0.sca_conv.style_bias
encoders.0.0.sca_conv.weight
encoders.0.0.sca_conv.bias
encoders.0.0.mod_conv4.style_weight
encoders.0.0.mod_conv4.style_bias
encoders.0.0.mod_conv4.weight
encoders.0.0.mod_conv4.bias
encoders.0.0.mod_conv5.style_weight
encoders.0.0.mod_conv5.style_bias
encoders.0.0.mod_conv5.weight
encoders.0.0.mod_conv5.bias
encoders.0.0.norm1.weight
encoders.0.0.norm1.bias
encoders.0.0.norm2.weight
encoders.0.0.norm2.bias
encoders.0.1.beta
encoders.0.1.gamma
encoders.0.1.mod_conv1.style_weight
en

: 